### Comment:
It was fun to do the Klotski challenge, but it's more efficient to just brute force Klotski. It took me less than half a second to find all 25955 possible positions (counting every similar block as equal). The shortest solution has 116 single steps and every configuration can be reached in 137 steps.

With recursion and backtracking, I got my solution in 2 seconds, but it required 3873 single steps (or 20928 steps when I forgot counting every similar block as equal).

#### commenting the comment:
when I ran the solver with the condition, that it should stop, if there is no new configuration reacheable, it stopped after 167 moves, so in my opinion every configuration can be reached in 167, not 137 steps, it was probably a typo...

In [1]:
import operator
import numpy as np
import sys

In [2]:
config = [[[0, 3], [0, 4]], 
          [[1, 3], [2, 3], [1, 4], [2, 4]],
          [[3, 3], [3, 4]],
          [[0, 1], [0, 2]],
          [[1, 2], [2, 2]],
          [[3, 1], [3, 2]],
          [[1, 1]],
          [[2, 1]],
          [[0, 0]],
          [[3, 0]]]

easy_config = [[[0, 3], [0, 4]],
               [[1, 1], [2, 1], [1, 2], [2, 2]],
               [[3, 3], [3, 4]],
               [[0, 1], [0, 2]],
               [[1, 4], [2, 4]],
               [[3, 1], [3, 2]],
               [[1, 3]],
               [[2, 3]],
               [[1, 0]],
               [[2, 0]]]

In [3]:
class Board:
    def __init__(self, board_state, previous_move):
        self.board_state = list(board_state)
        self.previous_move = previous_move
        self.children = []
        
    def check_solved(self):
        if self.board_state[1] == [[1, 0], [2, 0], [1, 1], [2, 1]]:
            return True
        return False
    
    def move(self, piece, direction):
        return [list(map(operator.add, point, direction)) for point in self.board_state[piece]]
    
    def move_possible(self, piece, direction):
        possible_piece = self.move(piece, direction)
        # außerhalb des Feldes
        if (True in [x_coordinate > 3 or x_coordinate < 0 for 
                     x_coordinate in [point[0] for point in possible_piece]] or 
            True in [y_coordinate > 4 or y_coordinate < 0 for 
                     y_coordinate in [point[1] for point in possible_piece]]):
            return False
        # Überlappung
        for point in possible_piece:
            if point in self.board_state[piece]:
                pass
            elif 1 in [block.count(point) for block in self.board_state]:
                return False
        return True
    
    def add_child(self, obj):
        self.children.append(obj)

In [4]:
def generate_children(board):
    for iter_piece in range(10):
        for direction in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
            if board.move_possible(iter_piece, direction): 
                child_state = list(board.board_state)
                child_state[iter_piece] = board.move(iter_piece, direction)
                new_move = list(board.previous_move)
                new_move.append((iter_piece, direction))
                board.add_child(Board(child_state, new_move))

In [5]:
def get_path(board):
    path = []
    for i in board.previous_move:
        if i[1] == [0, 1]:
            direction = 'up'
        elif i[1] == [0, -1]:
            direction = 'down'
        elif i[1] == [1, 0]:
            direction = 'right'
        elif i[1] == [-1, 0]:
            direction = 'left'
        path.append('piece %d moves %s' % (i[0], direction))
    return path

In [6]:
def build_and_check_tree(root):
    root = [root]
    current_layer = []
    history = [compress(root[0].board_state)]
    depth = 0
    while not any([board.check_solved() for board in root]):
        depth += 1
        for child in root:
            generate_children(child)
        for node in root:
            for child in node.children:
                if not compress(child.board_state) in history:
                    current_layer.append(child)
                    history.append(compress(child.board_state))
        print('depth: %d, boards in layer: %d' % (depth, len(current_layer)))
        root = list(current_layer)
        current_layer = []
    return root

In [7]:
def compress(input_pieces):
    len1 = []
    len2 = []
    len4 = []
    for i in range(len(input_pieces)):
        if len(input_pieces[i]) == 1:
            len1.append((input_pieces[i][0][0], input_pieces[i][0][1]))
            len1.sort(key = operator.itemgetter(0, 1))
        if len(input_pieces[i]) == 2:
            len2.append((input_pieces[i][0][0], input_pieces[i][0][1]))
            len2.sort(key = operator.itemgetter(0, 1))
        if len(input_pieces[i]) == 4:
            len4.append((input_pieces[i][0][0], input_pieces[i][0][1]))
    return [len1, len2, len4]

In [8]:
def draw_board(piece):
    board = np.full((5, 4), ' ')
    for i in range(10):
        # X-Wert
        for x in range(len(piece[i])):
            board[piece[i][x][1]][piece[i][x][0]] = i
    print(np.flip(np.fliplr(board)), "\n")

In [9]:
solutions = []
for i in build_and_check_tree(Board(config, [])):
    if i.check_solved():
        draw_board(i.board_state)
        solutions.append(i)

depth: 1, boards in layer: 4
depth: 2, boards in layer: 10
depth: 3, boards in layer: 13
depth: 4, boards in layer: 18
depth: 5, boards in layer: 16
depth: 6, boards in layer: 19
depth: 7, boards in layer: 20
depth: 8, boards in layer: 22
depth: 9, boards in layer: 32
depth: 10, boards in layer: 38
depth: 11, boards in layer: 42
depth: 12, boards in layer: 36
depth: 13, boards in layer: 34
depth: 14, boards in layer: 42
depth: 15, boards in layer: 40
depth: 16, boards in layer: 34
depth: 17, boards in layer: 30
depth: 18, boards in layer: 29
depth: 19, boards in layer: 36
depth: 20, boards in layer: 47
depth: 21, boards in layer: 60
depth: 22, boards in layer: 54
depth: 23, boards in layer: 48
depth: 24, boards in layer: 48
depth: 25, boards in layer: 39
depth: 26, boards in layer: 35
depth: 27, boards in layer: 45
depth: 28, boards in layer: 52
depth: 29, boards in layer: 68
depth: 30, boards in layer: 74
depth: 31, boards in layer: 69
depth: 32, boards in layer: 51
depth: 33, boards 

In [10]:
solutions = []
tree = Board(easy_config, [])
for i in build_and_check_tree(tree):
    if i.check_solved():
        draw_board(i.board_state)
        solutions.append(i)
        
i = tree
for j in range(2):
    i = i.children[0]
print(i.previous_move)

depth: 1, boards in layer: 4
depth: 2, boards in layer: 8
depth: 3, boards in layer: 9
[['0' '4' '4' '2']
 ['0' '6' '7' '2']
 ['3' ' ' ' ' '5']
 ['3' '1' '1' '5']
 ['8' '1' '1' '9']] 

[(3, [0, -1]), (0, [0, -1])]


In [11]:
num_lösungen = 0
for i in solutions:
    if i.check_solved():
        with open('Lösungen/Lösung%d.txt' % num_lösungen, 'w') as f:
            for item in get_path(i):
                f.write("%s\n" % item)
            num_lösungen += 1

### all possible configurations

In [16]:
def build_and_check_tree_all_config(root):
    root = [root]
    current_layer = []
    history = [compress(root[0].board_state)]
    depth = 0
    while root != []:
        depth += 1
        for child in root:
            generate_children(child)
        for node in root:
            for child in node.children:
                if not compress(child.board_state) in history:
                    current_layer.append(child)
                    history.append(compress(child.board_state))
                    draw_board(child.board_state)
        #print('depth: %d, boards in layer: %d' % (depth, len(current_layer)))
        root = list(current_layer)
        current_layer = []
    print(len(history))

In [ ]:
build_and_check_tree_all_config(Board(config, []))

In [15]:
with open("python.txt", "r") as input:
    python_list = input.read().split("\n\n")
    
with open("swift.txt", "r") as input:
    swift_list = input.read().split("\n\n")

In [16]:
len(python_list), len(swift_list)

(25823, 24358)

In [17]:
diff = []

for board in python_list:
    if not board in swift_list:
        diff.append(board)

In [18]:
swift_list

['GAAG\nGAAG\nGJJG\nG▢BG\nBB▢B',
 'GAAG\nGAAG\nGJJG\nGB▢G\nB▢BB',
 'GAAG\nGAAG\nGJJG\nGBBG\n▢B▢B',
 'GAAG\nGAAG\nGJJG\nGBBG\nB▢B▢',
 'GAAG\nGAAG\nGJJG\nG▢BG\nBB▢B',
 'GAAG\nGAAG\nGJJG\nGB▢G\nB▢BB',
 'GAAG\nGAAG\nGJJG\nGBBG\n▢▢BB',
 'GAAG\nGAAG\nGJJG\nGBBG\n▢BB▢',
 'GAAG\nGAAG\nGJJG\nGB▢G\n▢BBB',
 'GAAG\nGAAG\n▢JJG\nGBBG\nGB▢B',
 'GAAG\nGAAG\nGJJG\nGBBG\nBB▢▢',
 'GAAG\nGAAG\nGJJG\nG▢BG\nBBB▢',
 'GAAG\nGAAG\nGJJ▢\nGBBG\nB▢BG',
 'GAAG\nGAAG\nGJJG\nG▢BG\nB▢BB',
 'GAAG\nGAAG\nGJJG\nG▢▢G\nBBBB',
 'GAAG\nGAAG\nGJJG\nGB▢G\nBB▢B',
 'GAAG\nGAAG\nGJJG\nG▢BG\n▢BBB',
 'GAAG\nGAAG\n▢JJG\nGBBG\nG▢BB',
 'GAAG\nGAAG\n▢JJG\nGBBG\nGBB▢',
 'GAAG\nGAAG\nGJJ▢\nGBBG\n▢BBG',
 'GAAG\nGAAG\n▢JJG\nGB▢G\nGBBB',
 '▢AAG\nGAAG\nGJJG\nGBBG\nGB▢B',
 'GAAG\nGAAG\nJJ▢G\nGBBG\nGB▢B',
 'GAAG\nGAAG\nGJJG\nGB▢G\nBBB▢',
 'GAAG\nGAAG\nGJJ▢\nGBBG\nBB▢G',
 'GAAG\nGAAG\nGJJ▢\nG▢BG\nBBBG',
 'GAA▢\nGAAG\nGJJG\nGBBG\nB▢BG',
 'GAAG\nGAAG\nG▢JJ\nGBBG\nB▢BG',
 'GAAG\nGAAG\nG▢▢G\nGJJG\nBBBB',
 'GAAG\nGAAG\n▢JJG\nG▢BG\nGBBB',
 '▢AAG\nGA

In [19]:
python_list

['GAAG\nGAAG\nGJJG\nG▢BG\nBB▢B',
 'GAAG\nGAAG\nGJJG\nGB▢G\nB▢BB',
 'GAAG\nGAAG\nGJJG\nGBBG\n▢B▢B',
 'GAAG\nGAAG\nGJJG\nGBBG\nB▢B▢',
 'GAAG\nGAAG\nGJJG\nG▢BG\nB▢BB',
 'GAAG\nGAAG\nGJJG\nG▢▢G\nBBBB',
 'GAAG\nGAAG\nGJJG\nGB▢G\nBB▢B',
 'GAAG\nGAAG\nGJJG\nG▢BG\nBBB▢',
 'GAAG\nGAAG\nGJJG\nGB▢G\n▢BBB',
 'GAAG\nGAAG\n▢JJG\nGBBG\nGB▢B',
 'GAAG\nGAAG\nGJJG\nGBBG\n▢▢BB',
 'GAAG\nGAAG\nGJJG\nGBBG\n▢BB▢',
 'GAAG\nGAAG\nGJJ▢\nGBBG\nB▢BG',
 'GAAG\nGAAG\nGJJG\nGBBG\nBB▢▢',
 'GAAG\nGAAG\nGJJG\nG▢BG\n▢BBB',
 'GAAG\nGAAG\nG▢▢G\nGJJG\nBBBB',
 'GAAG\nGAAG\nGJJG\nGB▢G\nBBB▢',
 'GAAG\nGAAG\nGJJ▢\nG▢BG\nBBBG',
 'GAAG\nGAAG\n▢JJG\nGB▢G\nGBBB',
 '▢AAG\nGAAG\nGJJG\nGBBG\nGB▢B',
 'GAAG\nGAAG\nJJ▢G\nGBBG\nGB▢B',
 'GAAG\nGAAG\n▢JJG\nGBBG\nG▢BB',
 'GAAG\nGAAG\n▢JJG\nGBBG\nGBB▢',
 'GAAG\nGAAG\nGJJ▢\nGBBG\n▢BBG',
 'GAA▢\nGAAG\nGJJG\nGBBG\nB▢BG',
 'GAAG\nGAAG\nG▢JJ\nGBBG\nB▢BG',
 'GAAG\nGAAG\nGJJ▢\nGBBG\nBB▢G',
 'GAAG\nGAAG\n▢JJG\nG▢BG\nGBBB',
 'G▢▢G\nGAAG\nGAAG\nGJJG\nBBBB',
 'GAAG\nGAAG\nGJJ▢\nGB▢G\nBBBG',
 'GAA▢\nGA

In [20]:
for d in diff:
    print(d)
    print()

GAAG
GAAG
BGG▢
BGGB
▢JJB

GAAG
GAAG
▢GGB
BGGB
BJJ▢

GAA▢
GAAG
BGGG
BGGB
▢JJB

GAAG
GAAG
BGGB
BGG▢
▢JJB

GAAG
GAAG
BGG▢
▢GGB
BJJB

▢AAG
GAAG
GGGB
BGGB
BJJ▢

GAAG
GAAG
BGGB
▢GGB
BJJ▢

GAAG
GAAG
▢GGB
BGG▢
BJJB

GAA▢
GAAG
BGGG
▢GGB
BJJB

GAAG
GAAG
BGGB
▢GG▢
BJJB

GAAG
GAAG
BGGB
BGGB
▢JJ▢

GAAG
GAAG
▢GG▢
BGGB
BJJB

▢AAG
GAAG
GGGB
BGG▢
BJJB

GAA▢
GAAG
▢GGG
BGGB
BJJB

▢AAG
GAAG
GGG▢
BGGB
BJJB

▢AA▢
GAAG
GGGG
BGGB
BJJB

GGAA
GGAA
▢GG▢
BGGB
BJJB

AAGG
AAGG
▢GG▢
BGGB
BJJB

GGAA
GGAA
BGGB
▢GGB
BJJ▢

GGAA
GGAA
▢GGB
BGG▢
BJJB

▢GAA
GGAA
GGG▢
BGGB
BJJB

AAGG
AAGG
BGG▢
▢GGB
BJJB

AAGG
AAGG
BGGB
BGG▢
▢JJB

AAG▢
AAGG
▢GGG
BGGB
BJJB

GGAA
GGAA
BJJ▢
▢GGB
BGGB

AAGG
AAGG
▢JJB
BGG▢
BGGB

GGAA
GGAA
▢GGB
BGGB
BJJ▢

▢GAA
GGAA
GGGB
BGG▢
BJJB

AAG▢
AAGG
BGGG
▢GGB
BJJB

AAGG
AAGG
BGG▢
BGGB
▢JJB

GGAA
GGAA
BJJ▢
BGGB
▢GGB

GGAA
GGAA
BJJB
▢GG▢
BGGB

GGAA
GGAA
BJJB
BGGB
▢GG▢

AAGG
AAGG
▢JJB
BGGB
BGG▢

AAGG
AAGG
BJJB
▢GG▢
BGGB

AAGG
AAGG
BJJB
BGGB
▢GG▢

▢GAA
GGAA
GGGB
BGGB
BJJ▢

GAA▢
GAA▢
BBGG
BGGG
BGJJ

GAA▢
GAA▢
GB


GAAG
GAAG
BJJ▢
BGGB
BGG▢

AAG▢
AAGB
BJJB
▢GGG
BGGG

BBAA
BBAA
GGG▢
GGGG
▢JJG

BBAA
BBAA
GGGG
GGGG
JJ▢▢

AAGG
AAGG
BBGG
B▢GG
B▢JJ

AAGG
AAGG
B▢GG
BBGG
▢BJJ

AAGG
AAGG
▢BGG
BBGG
B▢JJ

AAGG
AAGG
BBGG
▢▢GG
BBJJ

AAGG
AAGG
▢▢GG
BBGG
BBJJ

AAGG
AAGG
▢BGG
▢BGG
BBJJ

AAGG
AAGG
BBGG
BBGG
▢▢JJ

AABB
AABB
▢GGG
GGGG
GJJ▢

AABB
AABB
GGGG
GGGG
▢▢JJ

GGAA
GGAA
GG▢B
GGBB
JJB▢

GGAA
GGAA
GGBB
GG▢B
JJ▢B

GGAA
GGAA
GGB▢
GGBB
JJ▢B

GGAA
GGAA
GG▢▢
GGBB
JJBB

GGAA
GGAA
GGBB
GG▢▢
JJBB

GGAA
GGAA
GGB▢
GGB▢
JJBB

GGAA
GGAA
GGBB
GGBB
JJ▢▢

GAAG
GAAG
JJ▢B
BGGB
▢GGB

▢AAG
GAAG
GJJB
BGGB
▢GGB

GAAG
GAAG
▢JJB
▢GGB
BGGB

GAA▢
GAAG
BJJG
BGGB
BGG▢

GAAG
GAAG
B▢JJ
BGGB
BGG▢

GAAG
GAAG
BJJ▢
BGG▢
BGGB

BBAA
BBAA
GGG▢
GGGG
JJ▢G

BBAA
BBAA
GG▢G
GGGG
JJG▢

AAGG
AAGG
BBGG
B▢GG
▢BJJ

AAGG
AAGG
BBGG
▢BGG
B▢JJ

AAGG
AAGG
▢BGG
BBGG
▢BJJ

▢▢GG
AAGG
AAGG
BBGG
BBJJ

AABB
AABB
▢GGG
GGGG
G▢JJ

AABB
AABB
G▢GG
GGGG
▢GJJ

GGAA
GGAA
GGBB
GG▢B
JJB▢

GGAA
GGAA
GGB▢
GGBB
JJB▢

GGAA
GGAA
GGBB
GGB▢
JJ▢B

GG▢▢
GGAA
GGAA
GGBB
JJBB

GAAG
GAAG
J